In [1]:
import numpy as np
import pandas as pd

In [2]:
test=pd.read_csv('test_tweets_anuFYb8.csv')
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [3]:
test.shape

(17197, 2)

In [4]:
train=pd.read_csv('train_E6oV3lV.csv')
train.shape

(31962, 3)

In [5]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [7]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [8]:
test['clean_tweets']=test['tweet'].apply(lambda x : clean_tweet(x))

In [9]:
print(test['clean_tweets'])

0        studiolife aislife requires passion dedication...
1        white supremacists want everyone to see the ne...
2        safe ways to heal your acne altwaystoheal heal...
3        is the hp and the cursed child book up for res...
4        3rd bihday to my amazing hilarious nephew eli ...
                               ...                        
17192    thought factory left right polarisation trump ...
17193    feeling like a mermaid hairflip neverready for...
17194    hillary campaigned today in ohio omg amp used ...
17195    happy at work conference right mindset leads t...
17196    my song so glad free download shoegaze newmusi...
Name: clean_tweets, Length: 17197, dtype: object


In [10]:
train['clean_tweets']=train['tweet'].apply(lambda x : clean_tweet(x))

In [11]:
print(train['clean_tweets'])

0        when a father is dysfunctional and is so selfi...
1        thanks for lyft credit i can t use cause they ...
2                                      bihday your majesty
3            model i love u take with u all the time in ur
4                        factsguide society now motivation
                               ...                        
31957                                   ate isz that youuu
31958    to see nina turner on the airwaves trying to w...
31959    listening to sad songs on a monday morning otw...
31960    sikh temple vandalised in in calgary wso conde...
31961                             thank you for you follow
Name: clean_tweets, Length: 31962, dtype: object


In [12]:
test['clean_tweets'] = test['clean_tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [13]:
train['clean_tweets'] = train['clean_tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [14]:
tokenized_tweet = test['clean_tweets'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [studiolife, aislife, requires, passion, dedic...
1    [white, supremacists, want, everyone, birds, m...
2    [safe, ways, heal, your, acne, altwaystoheal, ...
3    [cursed, child, book, reservations, already, w...
4    [bihday, amazing, hilarious, nephew, ahmir, un...
Name: clean_tweets, dtype: object

In [15]:
tokenized_tweet1 = train['clean_tweets'].apply(lambda x: x.split())
tokenized_tweet1.head()

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, lyft, credit, cause, they, offer, whe...
2                              [bihday, your, majesty]
3                      [model, love, take, with, time]
4                    [factsguide, society, motivation]
Name: clean_tweets, dtype: object

In [16]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [studiolif, aislif, requir, passion, dedic, wi...
1    [white, supremacist, want, everyon, bird, movi...
2    [safe, way, heal, your, acn, altwaystoh, healt...
3    [curs, child, book, reserv, alreadi, where, wh...
4    [bihday, amaz, hilari, nephew, ahmir, uncl, da...
Name: clean_tweets, dtype: object

In [17]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet1 = tokenized_tweet1.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet1.head()

0    [when, father, dysfunct, selfish, drag, kid, i...
1    [thank, lyft, credit, caus, they, offer, wheel...
2                              [bihday, your, majesti]
3                      [model, love, take, with, time]
4                          [factsguid, societi, motiv]
Name: clean_tweets, dtype: object

In [18]:
hypocrite = []
for i in range(len(train['tweet'])):
    if 'hypocrite' in train['tweet'][i]:
        if train['label'][i] == 1:
            hypocrite.append('racist')
        else:
            hypocrite.append('good')
    else:
        hypocrite.append('good')
df = pd.DataFrame(columns=['hypocrite'], data=hypocrite)
print(df['hypocrite'].value_counts())

train['hypocrite'] = hypocrite

good      31957
racist        5
Name: hypocrite, dtype: int64


In [19]:
X_train = train
y = train.label
X_test = test.clean_tweets

In [20]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [21]:
vectorizer = CountVectorizer(max_df=0.5)
#vectorizer1 = TfidfVectorizer(ngram_range=(1,3), max_df=0.5)

X = vectorizer.fit_transform(train['clean_tweets'])
X_test = vectorizer.transform(test['clean_tweets'])

In [22]:
model = LinearSVC(penalty='l2', C=0.5, dual=False, random_state=0, max_iter=1000)


In [23]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=0)

# calculate f1 score
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print('Accuracy:', accuracy_score(y_pred, y_val))
print("F1 Score: ", f1_score(y_pred, y_val))

Accuracy: 0.9662130455185359
F1 Score:  0.700831024930748


In [24]:
df = pd.DataFrame()
df['y_pred'] = y_pred
df['y_pred'].value_counts()

0    6079
1     314
Name: y_pred, dtype: int64

In [27]:
model.fit(X, y)
y_pred = model.predict(X_test)

In [28]:
df = pd.DataFrame()
df['y_pred'] = y_pred
df['y_pred'].value_counts()

0    16223
1      974
Name: y_pred, dtype: int64

In [29]:
submission=pd.read_csv("sample_submission_gfvA5FD.csv")

In [30]:
submission['label'] = y_pred
submission.to_csv('sub.csv', index=False)